In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

In [88]:
train = pd.read_csv("train_8wry4cB.csv")
test = pd.read_csv("test_Yix80N0.csv")

In [89]:
train.head()

,session_id,startTime,endTime,ProductList,gender
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,female
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,female
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,male


In [90]:
train.shape

(10500, 5)

In [91]:
test.head()

,session_id,startTime,endTime,ProductList
0,u12112,08/12/14 13:36,08/12/14 13:36,A00002/B00003/C00006/D19956/
1,u19725,19/12/14 13:52,19/12/14 13:52,A00002/B00005/C00067/D02026/
2,u11795,01/12/14 10:44,01/12/14 10:44,A00002/B00002/C00004/D12538/
3,u22639,08/12/14 20:19,08/12/14 20:22,A00002/B00003/C00079/D22781/;A00002/B00003/C00...
4,u18034,15/12/14 19:33,15/12/14 19:33,A00002/B00001/C00010/D23419/


In [92]:
test.shape

(4500, 4)

In [93]:
type(train)

pandas.core.frame.DataFrame

In [94]:
train.size

52500

In [95]:
train.tail()

,session_id,startTime,endTime,ProductList,gender
10495,u15442,18/11/14 7:39,18/11/14 7:42,A00002/B00002/C00007/D06407/;A00002/B00002/C00...,female
10496,u17986,25/11/14 15:16,25/11/14 15:16,A00006/B00030/C00334/D11660/,female
10497,u22508,09/12/14 10:11,09/12/14 10:11,A00002/B00002/C00007/D18028/,female
10498,u17087,22/11/14 11:27,22/11/14 11:27,A00003/B00012/C00131/D09453/;A00003/B00012/C00...,female
10499,u23137,19/12/14 3:11,19/12/14 3:19,A00002/B00001/C00010/D02309/;A00002/B00002/C00...,female


In [96]:
train['ProductList'][0]

'A00002/B00003/C00006/D28435/;A00002/B00003/C00006/D02554/;A00002/B00003/C00006/D28436/;A00002/B00003/C00006/D28437/'

In [97]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 5 columns):
session_id     10500 non-null object
startTime      10500 non-null object
endTime        10500 non-null object
ProductList    10500 non-null object
gender         10500 non-null object
dtypes: object(5)
memory usage: 410.3+ KB


In [98]:
train['gender'].value_counts()

female    8192
male      2308
Name: gender, dtype: int64

In [99]:
train.isnull().sum()

session_id     0
startTime      0
endTime        0
ProductList    0
gender         0
dtype: int64

In [100]:
#Custom function for extract infor from product list
def extract_product(str):
    if ";"  in str:
        prd_lst = str.split(";")
        count_item = len(prd_lst)
        first_lv1 = prd_lst[0].split("/")[0]
        first_lv2 = prd_lst[0].split("/")[1]
        lv1_lst =[]
        lv2_lst =[]
        for item in prd_lst:
            lv1_lst.append(item.split("/")[0])
            lv2_lst.append(item.split("/")[1])
        unique_lv1 = len(set(lv1_lst))
        unique_lv2 = len(set(lv2_lst))
        most_freq_lv1 =  max(lv1_lst, key=Counter(lv1_lst).get)        
    else:
        lv_lst = str.split("/")
        first_lv1 = lv_lst[0]
        first_lv2 = lv_lst[1]
        count_item = 1
        unique_lv1 = 1
        unique_lv2 = 1
        most_freq_lv1 = first_lv1
    return (count_item,first_lv1,first_lv2,unique_lv1,unique_lv2,most_freq_lv1)

In [101]:
#Feature Extraction :
new_col = ('NumProduct','FirstA','FirstB','UniqueA','UniqueB','MostA')      
new_col_lst = train['ProductList'].apply(lambda x: extract_product(x))    
new_col_df = pd.DataFrame(new_col_lst.tolist(),columns =new_col)

In [102]:
data = pd.concat([train, new_col_df], axis=1)

In [103]:
# Time feature extraction
data['startTime'] = pd.to_datetime(data['startTime'])
data['endTime'] = pd.to_datetime(data['endTime'])
data['duration'] = data['endTime'] - data['startTime']
data['duration'] = data['duration'].astype('timedelta64[m]')
data['weekday'] = data['startTime'].dt.dayofweek
data['hour_24h'] = data['startTime'].dt.hour

In [104]:
drop_lst = ['session_id', 'startTime', 'endTime', 'ProductList']

data = data.drop(drop_lst,axis =1 )

In [105]:
data.head()

,gender,NumProduct,FirstA,FirstB,UniqueA,UniqueB,MostA,duration,weekday,hour_24h
0,female,4,A00002,B00003,1,1,A00002,1.0,0,18
1,male,7,A00001,B00009,1,1,A00001,6.0,1,14
2,female,1,A00002,B00001,1,1,A00002,0.0,6,15
3,female,3,A00002,B00004,1,1,A00002,3.0,6,2
4,male,2,A00001,B00001,1,1,A00001,2.0,2,16


In [106]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()
data['gender'] = number.fit_transform(data['gender'].astype('str'))
data['FirstA'] = number.fit_transform(data['FirstA'].astype('str'))
data['FirstB'] = number.fit_transform(data['FirstB'].astype('str'))
data['MostA'] = number.fit_transform(data['MostA'].astype('str'))

In [107]:
data.head()

,gender,NumProduct,FirstA,FirstB,UniqueA,UniqueB,MostA,duration,weekday,hour_24h
0,0,4,1,2,1,1,1,1.0,0,18
1,1,7,0,8,1,1,0,6.0,1,14
2,0,1,1,0,1,1,1,0.0,6,15
3,0,3,1,3,1,1,1,3.0,6,2
4,1,2,0,0,1,1,0,2.0,2,16


In [108]:
test.isnull().sum().sum()

0

In [109]:
#Custom function for extract infor from product list
def extract_product(str):
    if ";"  in str:
        prd_lst = str.split(";")
        count_item = len(prd_lst)
        first_lv1 = prd_lst[0].split("/")[0]
        first_lv2 = prd_lst[0].split("/")[1]
        lv1_lst =[]
        lv2_lst =[]
        for item in prd_lst:
            lv1_lst.append(item.split("/")[0])
            lv2_lst.append(item.split("/")[1])
        unique_lv1 = len(set(lv1_lst))
        unique_lv2 = len(set(lv2_lst))
        most_freq_lv1 =  max(lv1_lst, key=Counter(lv1_lst).get)        
    else:
        lv_lst = str.split("/")
        first_lv1 = lv_lst[0]
        first_lv2 = lv_lst[1]
        count_item = 1
        unique_lv1 = 1
        unique_lv2 = 1
        most_freq_lv1 = first_lv1
    return (count_item,first_lv1,first_lv2,unique_lv1,unique_lv2,most_freq_lv1)

In [110]:
#Feature Extraction :
new_col = ('NumProduct','FirstA','FirstB','UniqueA','UniqueB','MostA')      
new_col_lst = test['ProductList'].apply(lambda x: extract_product(x))    
new_col_df = pd.DataFrame(new_col_lst.tolist(),columns =new_col)

In [111]:
test_data = pd.concat([test, new_col_df], axis=1)

In [112]:
# Time feature extraction
test_data['startTime'] = pd.to_datetime(test_data['startTime'])
test_data['endTime'] = pd.to_datetime(test_data['endTime'])
test_data['duration'] = test_data['endTime'] - test_data['startTime']
test_data['duration'] = test_data['duration'].astype('timedelta64[m]')
test_data['weekday'] = test_data['startTime'].dt.dayofweek
test_data['hour_24h'] = test_data['startTime'].dt.hour

In [113]:
drop_lst = ['session_id','startTime', 'endTime', 'ProductList']

test_data = test_data.drop(drop_lst,axis =1 )

In [114]:
test_data.head()

,NumProduct,FirstA,FirstB,UniqueA,UniqueB,MostA,duration,weekday,hour_24h
0,1,A00002,B00003,1,1,A00002,0.0,1,13
1,1,A00002,B00005,1,1,A00002,0.0,4,13
2,1,A00002,B00002,1,1,A00002,0.0,6,10
3,4,A00002,B00003,1,1,A00002,3.0,1,20
4,1,A00002,B00001,1,1,A00002,0.0,0,19


In [115]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()
#data['gender'] = number.fit_transform(data['gender'].astype('str'))
test_data['FirstA'] = number.fit_transform(test_data['FirstA'].astype('str'))
test_data['FirstB'] = number.fit_transform(test_data['FirstB'].astype('str'))
test_data['MostA'] = number.fit_transform(test_data['MostA'].astype('str'))

In [116]:
test_data.head()

,NumProduct,FirstA,FirstB,UniqueA,UniqueB,MostA,duration,weekday,hour_24h
0,1,1,2,1,1,1,0.0,1,13
1,1,1,4,1,1,1,0.0,4,13
2,1,1,1,1,1,1,0.0,6,10
3,4,1,2,1,1,1,3.0,1,20
4,1,1,0,1,1,1,0.0,0,19


In [117]:
data.shape

(10500, 10)

In [118]:
test_data.shape

(4500, 9)

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

In [176]:
# Devide data set into train set , test set
X = data.drop('gender', axis = 1)
Y = data.gender
#X_test = test_data.drop('session_id', axis = 1)

val_size = 0.30
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=val_size)

In [177]:
X_train.shape, X_val.shape, Y_train.shape

((7350, 9), (3150, 9), (7350,))

In [178]:
from sklearn.ensemble import RandomForestClassifier

In [179]:
clf = RandomForestClassifier()
clf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [180]:
Y_pred = clf.predict(test_data)
clf.score(X_train, Y_train)
acc_random_forest = round(clf.score(X_train, Y_train) * 100, 2)
acc_random_forest

94.39

In [181]:
import xgboost
classifier=xgboost.XGBClassifier(iterations = 100, max_depth=6, learning_rate=0.01, n_estimators=400)
classifier.fit(X_train, Y_train)
XY_pred = classifier.predict(test_data)
classifier.score(X_train, Y_train)
acc_xgboost = round(classifier.score(X_train, Y_train) * 100, 2)
acc_xgboost

88.18

In [184]:
# build the lightgbm model
import lightgbm as lgb
light = lgb.LGBMClassifier(boosting_type= 'gbdt', learning_rate= 0.003, objective='binary', num_leaves=10, max_depth=10)
light.fit(X_train, Y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.003, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=10, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [185]:
LY_pred = light.predict(test_data)
light.score(X_train, Y_train)
acc_lgboost = round(classifier.score(X_train, Y_train) * 100, 2)
acc_lgboost

88.18

In [159]:
from catboost import CatBoostClassifier

In [160]:
cbc = CatBoostClassifier(iterations = 1000, learning_rate=0.01, depth=6)
cbc.fit(X_train, Y_train)

0:	learn: 0.6859597	total: 36ms	remaining: 36s
1:	learn: 0.6788181	total: 48ms	remaining: 24s
2:	learn: 0.6717786	total: 60.2ms	remaining: 20s
3:	learn: 0.6648848	total: 82.8ms	remaining: 20.6s
4:	learn: 0.6581910	total: 98.3ms	remaining: 19.6s
5:	learn: 0.6516318	total: 111ms	remaining: 18.5s
6:	learn: 0.6451421	total: 124ms	remaining: 17.6s
7:	learn: 0.6390887	total: 140ms	remaining: 17.3s
8:	learn: 0.6329379	total: 153ms	remaining: 16.8s
9:	learn: 0.6269816	total: 164ms	remaining: 16.3s
10:	learn: 0.6210656	total: 176ms	remaining: 15.8s
11:	learn: 0.6154244	total: 184ms	remaining: 15.2s
12:	learn: 0.6099796	total: 195ms	remaining: 14.8s
13:	learn: 0.6044766	total: 204ms	remaining: 14.3s
14:	learn: 0.5995206	total: 211ms	remaining: 13.9s
15:	learn: 0.5946571	total: 217ms	remaining: 13.3s
16:	learn: 0.5897773	total: 226ms	remaining: 13s
17:	learn: 0.5850479	total: 233ms	remaining: 12.7s
18:	learn: 0.5800338	total: 243ms	remaining: 12.5s
19:	learn: 0.5752915	total: 252ms	remaining: 12.

172:	learn: 0.3740999	total: 1.59s	remaining: 7.6s
173:	learn: 0.3738889	total: 1.6s	remaining: 7.59s
174:	learn: 0.3737181	total: 1.61s	remaining: 7.58s
175:	learn: 0.3735172	total: 1.62s	remaining: 7.58s
176:	learn: 0.3732745	total: 1.63s	remaining: 7.57s
177:	learn: 0.3730835	total: 1.64s	remaining: 7.55s
178:	learn: 0.3728050	total: 1.64s	remaining: 7.53s
179:	learn: 0.3726420	total: 1.65s	remaining: 7.51s
180:	learn: 0.3724798	total: 1.66s	remaining: 7.5s
181:	learn: 0.3723006	total: 1.67s	remaining: 7.48s
182:	learn: 0.3720742	total: 1.67s	remaining: 7.47s
183:	learn: 0.3718546	total: 1.68s	remaining: 7.47s
184:	learn: 0.3716965	total: 1.69s	remaining: 7.45s
185:	learn: 0.3715195	total: 1.7s	remaining: 7.43s
186:	learn: 0.3714068	total: 1.7s	remaining: 7.41s
187:	learn: 0.3712025	total: 1.71s	remaining: 7.4s
188:	learn: 0.3709956	total: 1.72s	remaining: 7.38s
189:	learn: 0.3708037	total: 1.73s	remaining: 7.37s
190:	learn: 0.3707018	total: 1.74s	remaining: 7.36s
191:	learn: 0.3705

335:	learn: 0.3560946	total: 3.02s	remaining: 5.97s
336:	learn: 0.3560346	total: 3.03s	remaining: 5.97s
337:	learn: 0.3559892	total: 3.04s	remaining: 5.96s
338:	learn: 0.3559154	total: 3.06s	remaining: 5.96s
339:	learn: 0.3558585	total: 3.07s	remaining: 5.95s
340:	learn: 0.3557452	total: 3.08s	remaining: 5.95s
341:	learn: 0.3557163	total: 3.12s	remaining: 6s
342:	learn: 0.3556293	total: 3.13s	remaining: 6s
343:	learn: 0.3555106	total: 3.15s	remaining: 6s
344:	learn: 0.3554418	total: 3.15s	remaining: 5.99s
345:	learn: 0.3553772	total: 3.17s	remaining: 5.98s
346:	learn: 0.3553403	total: 3.2s	remaining: 6.02s
347:	learn: 0.3552892	total: 3.21s	remaining: 6.01s
348:	learn: 0.3552234	total: 3.21s	remaining: 6s
349:	learn: 0.3551772	total: 3.22s	remaining: 5.98s
350:	learn: 0.3551056	total: 3.23s	remaining: 5.97s
351:	learn: 0.3550262	total: 3.24s	remaining: 5.96s
352:	learn: 0.3549517	total: 3.24s	remaining: 5.95s
353:	learn: 0.3549218	total: 3.25s	remaining: 5.93s
354:	learn: 0.3548899	tot

503:	learn: 0.3476156	total: 4.76s	remaining: 4.69s
504:	learn: 0.3475870	total: 4.77s	remaining: 4.68s
505:	learn: 0.3475241	total: 4.78s	remaining: 4.67s
506:	learn: 0.3474257	total: 4.79s	remaining: 4.66s
507:	learn: 0.3474024	total: 4.8s	remaining: 4.65s
508:	learn: 0.3473432	total: 4.81s	remaining: 4.64s
509:	learn: 0.3472693	total: 4.82s	remaining: 4.63s
510:	learn: 0.3472229	total: 4.83s	remaining: 4.62s
511:	learn: 0.3471862	total: 4.84s	remaining: 4.61s
512:	learn: 0.3471569	total: 4.84s	remaining: 4.6s
513:	learn: 0.3470625	total: 4.86s	remaining: 4.59s
514:	learn: 0.3469844	total: 4.86s	remaining: 4.58s
515:	learn: 0.3469425	total: 4.87s	remaining: 4.57s
516:	learn: 0.3469208	total: 4.88s	remaining: 4.56s
517:	learn: 0.3468992	total: 4.89s	remaining: 4.55s
518:	learn: 0.3468594	total: 4.9s	remaining: 4.54s
519:	learn: 0.3468007	total: 4.91s	remaining: 4.53s
520:	learn: 0.3467396	total: 4.92s	remaining: 4.53s
521:	learn: 0.3467094	total: 4.94s	remaining: 4.52s
522:	learn: 0.3

665:	learn: 0.3416322	total: 6.32s	remaining: 3.17s
666:	learn: 0.3416036	total: 6.33s	remaining: 3.16s
667:	learn: 0.3415853	total: 6.33s	remaining: 3.15s
668:	learn: 0.3415758	total: 6.34s	remaining: 3.14s
669:	learn: 0.3415644	total: 6.35s	remaining: 3.13s
670:	learn: 0.3415230	total: 6.36s	remaining: 3.12s
671:	learn: 0.3414899	total: 6.37s	remaining: 3.11s
672:	learn: 0.3414588	total: 6.38s	remaining: 3.1s
673:	learn: 0.3414168	total: 6.39s	remaining: 3.09s
674:	learn: 0.3413658	total: 6.4s	remaining: 3.08s
675:	learn: 0.3413225	total: 6.41s	remaining: 3.07s
676:	learn: 0.3412910	total: 6.42s	remaining: 3.06s
677:	learn: 0.3412648	total: 6.43s	remaining: 3.05s
678:	learn: 0.3412443	total: 6.45s	remaining: 3.05s
679:	learn: 0.3412209	total: 6.46s	remaining: 3.04s
680:	learn: 0.3412095	total: 6.46s	remaining: 3.03s
681:	learn: 0.3411486	total: 6.49s	remaining: 3.02s
682:	learn: 0.3410962	total: 6.5s	remaining: 3.02s
683:	learn: 0.3410846	total: 6.51s	remaining: 3.01s
684:	learn: 0.3

824:	learn: 0.3362460	total: 7.75s	remaining: 1.64s
825:	learn: 0.3361956	total: 7.75s	remaining: 1.63s
826:	learn: 0.3361702	total: 7.76s	remaining: 1.62s
827:	learn: 0.3361233	total: 7.77s	remaining: 1.61s
828:	learn: 0.3361084	total: 7.78s	remaining: 1.6s
829:	learn: 0.3360536	total: 7.79s	remaining: 1.59s
830:	learn: 0.3360372	total: 7.79s	remaining: 1.58s
831:	learn: 0.3359931	total: 7.8s	remaining: 1.57s
832:	learn: 0.3359267	total: 7.81s	remaining: 1.56s
833:	learn: 0.3358819	total: 7.82s	remaining: 1.55s
834:	learn: 0.3358510	total: 7.82s	remaining: 1.55s
835:	learn: 0.3358177	total: 7.83s	remaining: 1.54s
836:	learn: 0.3357985	total: 7.84s	remaining: 1.53s
837:	learn: 0.3357251	total: 7.85s	remaining: 1.52s
838:	learn: 0.3357026	total: 7.86s	remaining: 1.51s
839:	learn: 0.3356889	total: 7.87s	remaining: 1.5s
840:	learn: 0.3356544	total: 7.88s	remaining: 1.49s
841:	learn: 0.3356217	total: 7.89s	remaining: 1.48s
842:	learn: 0.3356096	total: 7.9s	remaining: 1.47s
843:	learn: 0.33

994:	learn: 0.3301336	total: 9.45s	remaining: 47.5ms
995:	learn: 0.3301134	total: 9.46s	remaining: 38ms
996:	learn: 0.3300562	total: 9.47s	remaining: 28.5ms
997:	learn: 0.3300230	total: 9.49s	remaining: 19ms
998:	learn: 0.3299949	total: 9.49s	remaining: 9.5ms
999:	learn: 0.3299481	total: 9.5s	remaining: 0us


In [161]:
CY_pred = cbc.predict(test_data)
cbc.score(X_train, Y_train)
acc_caboost = round(classifier.score(X_train, Y_train) * 100, 2)
acc_caboost

88.07

In [126]:
pred=pd.DataFrame(XY_pred)
sub_df=pd.read_csv('sample_submission_opxHi4g.csv')

In [127]:
sub_df

,session_id,gender
0,u12112,female
1,u19725,male
2,u11795,female
3,u22639,female
4,u18034,male
...,...,...
4495,u23966,male
4496,u20527,female
4497,u13253,male
4498,u17094,male


In [128]:
pred.shape

(4500, 1)

In [129]:
datasets=pd.concat([sub_df['session_id'],pred],axis=1)
datasets.columns=['session_id','gender']

In [130]:
datasets.to_csv('sample_submit.csv',index=False)

In [131]:
datasets

,session_id,gender
0,u12112,0
1,u19725,0
2,u11795,0
3,u22639,0
4,u18034,0
...,...,...
4495,u23966,1
4496,u20527,0
4497,u13253,0
4498,u17094,1
